In [ ]:
!pip install tweepy

In [ ]:
!pip install ntscraper

In [ ]:
import tweepy
from ntscraper import Nitter
from google.colab import drive

# Step 1: Connect to Google Drive
drive.mount('/content/drive')

# Path to the replied_id.txt file
replied_id_file = '/content/drive/My Drive/x_bot/replied_id.txt'

# Step 2: Initialize the tweepy Client with your Twitter application credentials
client = tweepy.Client(consumer_key="YOUR_CONSUMER_KEY",
                       consumer_secret="YOUR_CONSUMER_SECRET",
                       access_token="YOUR_ACCESS_TOKEN",
                       access_token_secret="YOUR_ACCESS_TOKEN_SECRET")

# Step 3: Read the replied_id.txt file
def read_replied_ids(file_path):
    try:
        with open(file_path, 'r') as file:
            replied_ids = file.read().splitlines()
            return replied_ids
    except FileNotFoundError:
        return []

# Step 4: Scrape tweets based on search terms
scraper = Nitter()
search_terms = ["term1", "term2"]
tweet_ids = []

for term in search_terms:
    posts = scraper.get_tweets(term, mode='term', number=1000)
    for post in posts['tweets']:
        if post['stats']['likes'] > 100:
            link = post['link']
            tweet_id = link.split('/')[-1].split('#')[0]
            tweet_ids.append(tweet_id)

# Step 5: Filter out already replied tweets
replied_ids = read_replied_ids(replied_id_file)
new_tweet_ids = [tweet_id for tweet_id in tweet_ids if tweet_id not in replied_ids]

# Step 6: Reply to new tweets and update the replied_id.txt file
if new_tweet_ids:
    with open(replied_id_file, 'a') as file:
        for tweet_id in new_tweet_ids:
            try:
                response = client.create_tweet(text="YOUR_REPLY_TEXT", in_reply_to_tweet_id=tweet_id)
                print(f"Reply posted successfully to tweet ID: {tweet_id}")
                file.write(tweet_id + '\n')
            except Exception as e:
                print(f"Failed to reply to tweet ID: {tweet_id}. Error: {e}")
else:
    print("No new tweets to reply to.")
